# Some interesting things from VicRoads Crash Stats

In [40]:
# getting this book to run
# you may need to change the iopub_data_rate_limit as per https://github.com/jupyter/notebook/issues/2287

all of this code will work with plot.ly offline, EXCEPT getting images to appear in the PDF output.
super annoyingly, they don't support this in offline mode yet

on first run, you need to run:
```
import plotly 
plotly.tools.set_credentials_file(username='DemoAccount', api_key='thisisakey')
```

with a plotly username and api key

this will only affect running the line to save and show plits - you can replace these with

```
py.iplot(figure1, filename = 'foo')
```

if you do not need to be able to save the notebook as a PDF

import data, check it read correctly

In [41]:
import pandas as pd

df_accident = pd.read_csv(
    'vicroads_data/ACCIDENT/ACCIDENT.csv', # file name and path
    sep=',', # separated by commas
    index_col=0 # first column should be primary key
)

In [42]:
# data read okay?
print(df_accident.head(1))

             ACCIDENTDATE ACCIDENTTIME  ACCIDENT_TYPE      Accident Type Desc  \
ACCIDENT_NO                                                                     
T20060000010   13/01/2006     12.42.00              1  Collision with vehicle   

              DAY_OF_WEEK Day Week Description  DCA_CODE  \
ACCIDENT_NO                                                
T20060000010            6               Friday       113   

                                             DCA Description DIRECTORY  \
ACCIDENT_NO                                                              
T20060000010  RIGHT NEAR (INTERSECTIONS ONLY)                      MEL   

              EDITION     ...     NO_PERSONS NO_PERSONS_INJ_2  \
ACCIDENT_NO               ...                                   
T20060000010     40.0     ...              6                0   

              NO_PERSONS_INJ_3  NO_PERSONS_KILLED NO_PERSONS_NOT_INJ  \
ACCIDENT_NO                                                            
T20060000010

In [43]:
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.plotly as pyonline
from plotly.offline import *
import plotly.graph_objs as go
#import matplotlib as mpl
from IPython.display import Image
init_notebook_mode(connected=True) # render plotly charts in the notebook on the fly
#init_notebook_mode()

Code below to manipulate data: create a copy of df_accident, add columns for:
* Year
* Month
* Hour

Filter out 2006 and 2017, as they are incomplete years in this data

In [44]:
# Manipulate data
df_accident_mod = df_accident
#print(df_accident_mod.head(3))

## DATES

# make a string representing date time
df_accident_mod['date'] = df_accident_mod['ACCIDENTDATE'] + ' ' + df_accident_mod['ACCIDENTTIME']

# convert to datetime
df_accident_mod['date'] = pd.to_datetime(df_accident_mod['date'], format="%d/%m/%Y %H.%M.%S")

# make some useful date fields
df_accident_mod['year'] = pd.DatetimeIndex(df_accident_mod['date']).year
df_accident_mod['month'] = pd.DatetimeIndex(df_accident_mod['date']).month
df_accident_mod['hour'] = pd.DatetimeIndex(df_accident_mod['date']).hour

#print(df_accident_mod.head(3))

## FILTER

# remove < 2007 and > 2016, as 2006 and 2017 are incomplete years
df_accident_mod = df_accident_mod[df_accident_mod['year'] >= 2007]
df_accident_mod = df_accident_mod[df_accident_mod['year'] <= 2016]

# remove speed limits > 110 km/h - 777, 888, and 999 are used to code:
# 777 = other
# 888 = camping grounds or off road
# 999 = not known

df_accident_mod = df_accident_mod[df_accident_mod['SPEED_ZONE'] <= 110]

print(df_accident_mod.head(1))

             ACCIDENTDATE ACCIDENTTIME  ACCIDENT_TYPE  \
ACCIDENT_NO                                             
T20070000004    1/01/2007     02.55.00              4   

                         Accident Type Desc  DAY_OF_WEEK Day Week Description  \
ACCIDENT_NO                                                                     
T20070000004  Collision with a fixed object            2               Monday   

              DCA_CODE                                    DCA Description  \
ACCIDENT_NO                                                                 
T20070000004       171  LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...   

             DIRECTORY  EDITION  ...  NO_PERSONS_NOT_INJ POLICE_ATTEND  \
ACCIDENT_NO                      ...                                     
T20070000004       VCS      9.0  ...                   0             1   

              ROAD_GEOMETRY   Road Geometry Desc SEVERITY  SPEED_ZONE  \
ACCIDENT_NO                                             

Code to group by year in Pandas

In [83]:
accidents_by_year =  df_accident_mod.groupby(['year']).size().reset_index(name="cnt")
print(accidents_by_year.head(3))

   year    cnt
0  2007  12980
1  2008  13494
2  2009  13097


Code to make the chart series and layout

In [46]:
# setup series and chart
seriesCrashesbyYearBar = go.Bar(
    x = accidents_by_year['year'],
    y = accidents_by_year['cnt'],
    name = 'Crashes per Year',
    marker = dict(
        color = '#ee2737'
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents per Year',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 1, # label every 1 bars
        title = 'Year',
            titlefont=dict(
                family='Open Sans',
                size=16
            )
    ),
    yaxis = dict(
        title = 'Count',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    )
)

data = [seriesCrashesbyYearBar] # can be multiple series
figure1 = go.Figure(data=data, layout=layout)

# to just show in notebook
# py.iplot(figure1, filename = 'figure1')

# to use the online thing to save static images
#pyonline.image.save_as(figure1, filename='img/figure1.png')
# and to show them
#Image('img/figure1.png')

In [84]:
# to be able to export to image and show as image, so this will all load in a nice exported pdf
#pyonline.image.save_as(figure1, filename='img/figure1.png')

#from IPython.display import Image
#Image('a-simple-plot.png')

# display as image in notebook, for saving to pdf
#pyonline.image.ishow(figure1, width=1500, height=750)

## Number of accidents over time

* The number of accidents per year has been relatively steady for the last decade

In [57]:
py.iplot(figure1, filename = 'figure1')
#pyonline.image.ishow(figure1, width=1500, height=750)

In [61]:
accidents_by_speed =  df_accident_mod.groupby([ 'SPEED_ZONE']).size().reset_index(name="cnt")
print(accidents_by_speed.head(3))

speedLimitZoneSeries = go.Bar(
    y = accidents_by_speed['cnt'],
    x = accidents_by_speed['SPEED_ZONE'],
    name = 'Num. Crashes',
    marker = dict(
        color = '#ee2737'
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents by Speed Limit Zone',
    titlefont=dict(
            family='Open Sans',
            size=22#,
            #color='#7f7f7f'
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 10, # label every 1 bars
        title = 'Speed Limit Zone',
            titlefont=dict(
                family='Open Sans',
                size=16#,
                #color='#7f7f7f'
            )
    ),
    yaxis = dict(
        title = 'Count',
        titlefont=dict(
            family='Open Sans',
            size=16#,
            #color='#7f7f7f'
        )
    )
)

data = [speedLimitZoneSeries] # can be multiple series

figure2 = go.Figure(data=data, layout=layout)

   SPEED_ZONE    cnt
0          30    170
1          40   5388
2          50  25395


## Most accidents happen at lower speeds

* Almost twice as many crashes happen in 60 km/h zones than in any other

In [62]:
py.iplot(figure2, filename='figure2')
#pyonline.image.ishow(figure2, width=1500, height=750)

In [85]:
# group by severity and speed zone
accidents_by_speed_by_severity =  df_accident_mod.groupby(['SPEED_ZONE','SEVERITY']).size().reset_index(name="cnt")
#print(accidents_by_speed_by_severity.head(3))

# only keep severe
accidents_by_speed_severity_1 = accidents_by_speed_by_severity[accidents_by_speed_by_severity['SEVERITY'] == 1]
#print(accidents_by_speed_severity_1.head(3))

speedLimitZoneSeriesSevere = go.Bar(
    y = accidents_by_speed_severity_1['cnt'],
    x = accidents_by_speed_severity_1['SPEED_ZONE'],
    name = 'Severe Crashes by Speed Limit Zone',
    marker = dict(
        color = '#0077c8'
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents by Speed Limit Zone',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 10, # label every 1 bars
        title = 'Speed Limit Zone',
            titlefont=dict(
                family='Open Sans',
                size=16
            )
    ),
    yaxis = dict(
        title = 'Count',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    )
)

data = [speedLimitZoneSeries, speedLimitZoneSeriesSevere] # can be multiple series

figure3 = go.Figure(data=data, layout=layout)

## The vast majority of crashes are not lethal, but lethality increases with the posted speed limit

* You can see below that more lethal crashes happen in 100 zones than 60 zones, even though there are less than hlaf as many accidents
* This is presumably because more accients happen at higher speeds in higher limit zone
* This is also likely a proxy for higher speed limits representing country roads

In [64]:
py.iplot(figure3, filename='figure3')
#pyonline.image.ishow(figure3, width=1500, height=750)

Proportion of crashes that are lethal by speed limit zone

This snippet of code:
* Groups by both Speed Zone and Severity
* Applys a function to calculate a fields percentage of the first group by field (speed zone), so all speed zone 30 sum to 100, all speed zone 40 sum to 100, etc
* filters for just severe crashes (severity = 1), indicating someone died

In [67]:
# group by severity and speed zone
accidents_by_speed_by_severity =  df_accident_mod.groupby(['SPEED_ZONE','SEVERITY']).size()
# divide by the first level you grouped by, multiply by 100, to end up with each severitys % of speed zone crashes
accidents_by_speed_by_severity_pc = accidents_by_speed_by_severity.groupby(level=0).apply(lambda x:round(
                                                 100 * x / float(x.sum()),2))
# reset index and name the new column count
accidents_by_speed_by_severity_pc = accidents_by_speed_by_severity_pc.reset_index(name="count")

# filter for just severe
accidents_by_speed_severe_pc = accidents_by_speed_by_severity_pc[accidents_by_speed_by_severity_pc['SEVERITY'] == 1]

# drop now-redundant column
del accidents_by_speed_severe_pc['SEVERITY']

print(accidents_by_speed_severe_pc.head(3))

   SPEED_ZONE  count
0          30   0.59
3          40   0.45
6          50   0.87


In [302]:
speedLimitZoneSeriesSeverePc = go.Scatter(
    y = accidents_by_speed_severe_pc['count'],
    x = accidents_by_speed_severe_pc['SPEED_ZONE'],
    name = '% Crashes Lethal',
    marker = dict(
        color = '#0077c8'
    ),
    yaxis='y2'
)

layout = go.Layout(
    barmode = 'group',
    title = 'VicRoads Accident - Accidents by Speed Limit Zone',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 10, # label every 10th number after 0
        title = 'Speed Limit Zone',
            titlefont=dict(
                family='Open Sans',
                size=16
            )
    ),
    yaxis = dict(
        title = 'Num. Crashes',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        dtick=5000,
        range=[0,65000]
    ),
    yaxis2=dict(
        title='% Crashes Lethal',
        overlaying='y',
        side='right',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        range=[0,6.5],
        dtick = 1
    )
)

data = [speedLimitZoneSeries, speedLimitZoneSeriesSeverePc] # can be multiple series

figure4 = go.Figure(data=data, layout=layout)

## Lethality as a proportion of accidents increases with Speed Limit Zone

* While most accidents happen at slower speeds, accidents in higher-speed zones are more likely to be deadly

In [303]:
py.iplot(figure4, filename='figure4')
#pyonline.image.ishow(figure4, width=1500, height=750)

### Accidents by time of day

In [70]:
accidents_by_hour = df_accident_mod.groupby(['hour']).size().reset_index(name="cnt")
print(accidents_by_hour.head(3))

   hour   cnt
0     0  2067
1     1  1755
2     2  1415


In [72]:
seriesAccidentsByHourBar = go.Bar(
    y = accidents_by_hour['cnt'],
    x = accidents_by_hour['hour'],
    name = 'Num. Crashes',
    marker = dict(
        color = '#FF6900'
    )
)

layout = go.Layout(
    barmode = 'group',
    title = 'VicRoads Accident - Accidents by Hour',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        #tickangle = -45, # angle lables at 45 deg
        dtick = 1,
        title = 'Speed Limit Zone',
            titlefont=dict(
                family='Open Sans',
                size=16
            )
    ),
    yaxis = dict(
        title = 'Num. Crashes',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    )
)

data = [seriesAccidentsByHourBar]

figure5 = go.Figure(data=data, layout=layout)

## Most accidents happen during daylight hours

* This is presumably reflecting when more people are on the road
* More accidents happen in the afternoon than the morning


In [73]:
py.iplot(figure5, filename='figure5')
#pyonline.image.ishow(figure5, width=1500, height=750)

crashes by time by severity

In [74]:
# group by hour by severity
accidents_by_hour_by_severity =  df_accident_mod.groupby(['hour','SEVERITY']).size()

# divide by the first level you grouped by, multiply by 100, to end up with each severitys % of speed zone crashes
accidents_by_hour_by_severity_pc = accidents_by_hour_by_severity.groupby(level=0).apply(lambda x:round(
                                                 100 * x / float(x.sum()),2))
# reset index and name the new column count
accidents_by_hour_by_severity_pc = accidents_by_hour_by_severity_pc.reset_index(name="count")

# filter for just severe
accidents_by_hour_by_severity_1 = accidents_by_hour_by_severity_pc[accidents_by_hour_by_severity_pc['SEVERITY'] == 1]

# drop now-redundant column
del accidents_by_hour_by_severity_1['SEVERITY']

print(accidents_by_hour_by_severity_1.head(3))

   hour  count
0     0   3.97
3     1   4.33
6     2   4.59


In [75]:
seriesAccidentsByHourSevereLine = go.Scatter(
    y = accidents_by_hour_by_severity_1['count'],
    x = accidents_by_hour_by_severity_1['hour'],
    name = 'Num. Crashes',
    marker = dict(
        color = '#0077C8'
    ),
    yaxis = 'y2'
)

layout = go.Layout(
    barmode = 'group',
    title = 'VicRoads Accident - Accidents by Hour - Lethal vs All',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        title = 'Speed Limit Zone',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    yaxis = dict(
        title = 'Num. Crashes',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        range=[0,12000]
    ),
    yaxis2=dict(
        title='% Crashes Lethal',
        overlaying='y',
        side='right',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        range=[0,6],
        dtick = 1
    )
)

data = [seriesAccidentsByHourBar, seriesAccidentsByHourSevereLine]

figure6 = go.Figure(data=data, layout=layout)

## Accidents at night are more likley to be lethal

* The most accidents occur between 5 and 6pm, this also has the lowest mortality rate, with slightly more than 1 in 100 accidents leading to a death
* The least accidents happen between 2 and 4 am, where the rate of accidents leading to death is approximately 3 times higher than at 5pm, around 4.5 in 10)

In [76]:
py.iplot(figure6, filename='figure6')
#pyonline.image.ishow(figure6, width=1500, height=750)

### Accidents weighted for Population change

The number of accidents looks pretty stable over this period (see figure 1). But Victoria has grown a lot in the last decade - Have more or less crashes happened per person?

To estimate, below we:
* Import the ABS ERP dataset for Victoria, code 3101.0
    * Available at: http://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/3101.0Jun%202016?OpenDocument
* Tidy that up, and average by year (as it is quarterly)
* Join to crash dataset, and divide crashes by population to estimate crashes per population

#### Get ERP Data

In [91]:
# read from ABS Excel series
df_erp = pd.read_excel(
    'abs_data/erp_2016.xlsx', # file name and path
    sheetname = 'Data1',
    header = 0,
    skiprows = 9,
    index_col=0 # first column should be primary key
)

# just keep the Victoria Persons series, which has the id A2060844K
df_erp = df_erp['A2060844K']

# fix column headers
df_erp = df_erp.reset_index()

# rename columns
df_erp.columns = ['date','erp']

# add a year column
df_erp['year'] = pd.DatetimeIndex(df_erp['date']).year

print(df_erp.head(3))

# group by year
erp_by_year = df_erp.groupby(['year']).mean().round()

# fix column headers
erp_by_year = erp_by_year.reset_index()

print(erp_by_year.head(3))

        date      erp  year
0 1981-06-01  3946917  1981
1 1981-09-01  3957333  1981
2 1981-12-01  3968398  1981
   year        erp
0  1981  3957549.0
1  1982  3997278.0
2  1983  4040160.0


#### Merge ERP data with Crash data

In [90]:
# dataframe accidents_by_year defined earlier in the file
print(accidents_by_year.head(3))
print(erp_by_year.head(3))

# take all rows from accidents by year, left join with erp_by_year on year=year
df_accidents_by_year_with_erp = pd.merge(accidents_by_year, erp_by_year, on='year', how='left')

# did we get what we expect?
print(df_accidents_by_year_with_erp.head(3))

# make column for crashes per thousand people
df_accidents_by_year_with_erp['crashes_per_100k'] = round(
    100000 * (
        df_accidents_by_year_with_erp['cnt']/df_accidents_by_year_with_erp['erp']
    ),0
)

   year    cnt
0  2007  12980
1  2008  13494
2  2009  13097
   year        erp
0  1981  3957549.0
1  1982  3997278.0
2  1983  4040160.0
   year    cnt        erp
0  2007  12980  5166404.0
1  2008  13494  5272302.0
2  2009  13097  5384432.0


In [93]:
seriesCrashesbyYearBy100KPopulationScatter = go.Scatter(
    y = df_accidents_by_year_with_erp['crashes_per_100k'],
    x = df_accidents_by_year_with_erp['year'],
    name = 'Crashes Per 100k Population Per Year',
    marker = dict(
        color = '#0077C8',
        size=10
    ),
    yaxis = 'y2',
    line = dict(
        color = '#0077C8',
        width=4
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Crashes by Year',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        title = 'Year',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    yaxis = dict(
        title = 'Num. Crashes',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        range=[0,15000],
        dtick = 2500
    ),
    yaxis2=dict(
        title='Crashes per 100k Popn',
        overlaying='y',
        side='right',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        range=[0,300]
    ),
    legend=dict(
        orientation="h",
        x=.24,
        y=1.1
    )
)

data = [seriesCrashesbyYearBar, seriesCrashesbyYearBy100KPopulationScatter]

figure7 = go.Figure(data=data, layout=layout)

## Accidents are steadily declining, weighted for population

While the number of accidents has remained relatively steady, the number of acidents per 100,000 people has fallen from a peak of 256 per 100,000 in 2008 to 208 in 2016, a fall of 19%

In [94]:
py.iplot(figure7, filename='figure7')
#pyonline.image.ishow(figure7, width=1500, height=750)

## Accidents by Location

Below we work through some analysis of accidents by local government area

In [97]:
df_node = pd.read_csv(
    'vicroads_data/ACCIDENT/NODE.csv', # file name and path
    sep=','#, # separated by commas
    #index_col=0, # first column should be primary key
    #low_memory = False
)

# filter out all values in the LGA_NAME column that don't represent LGAs
# read valid lga list
valid_lgas = pd.read_csv(
    'vic_lgas.csv',
    names=['LGA']                        
)

# make upper case to match vicroads data
valid_lgas['LGA'] = valid_lgas['LGA'].str.upper()

# convert to a list
valid_lgas = valid_lgas['LGA'].values.tolist()

# normalise LGA names
df_node['LGA_NAME'].replace('DANDENONG', 'GREATER DANDENONG',inplace=True)
df_node['LGA_NAME'].replace('GEELONG', 'GREATER GEELONG',inplace=True)
df_node['LGA_NAME'].replace('COLAC OTWAY', 'COLAC-OTWAY',inplace=True)
df_node['LGA_NAME'].replace('SHEPPARTON', 'GREATER SHEPPARTON',inplace=True)
df_node['LGA_NAME'].replace('BENDIGO', 'GREATER BENDIGO',inplace=True)

# filter where name not in list
df_node = df_node[df_node['LGA_NAME'].isin(valid_lgas)]

# merge node columns onto limited dataset
df_tmp = df_accident_mod.reset_index()
df_accident_node = pd.merge(df_tmp, df_node, on='ACCIDENT_NO', how='left')

print(df_accident_node.head(1))

    ACCIDENT_NO ACCIDENTDATE ACCIDENTTIME  ACCIDENT_TYPE  \
0  T20070000004    1/01/2007     02.55.00              4   

              Accident Type Desc  DAY_OF_WEEK Day Week Description  DCA_CODE  \
0  Collision with a fixed object            2               Monday       171   

                                     DCA Description DIRECTORY     ...       \
0  LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...       VCS     ...        

   NODE_TYPE        AMG_X        AMG_Y  LGA_NAME  Lga Name All  \
0          N  2477943.291  2596972.868  CAMPASPE      CAMPASPE   

       Region Name           Deg Urban Name       Lat       Long  Postcode No  
0  NORTHERN REGION  Small Provincial Cities -36.12578  144.75497       3564.0  

[1 rows x 43 columns]


count all accidents by LGA

In [98]:
# group by hour by severity
accidents_by_lga =  df_accident_node.groupby(['LGA_NAME']).size()

# reset index and name the new column count
accidents_by_lga = accidents_by_lga.reset_index(name="count")

### The LGAs with the highest accident rates:

In [99]:
accidents_by_lga.nlargest(5,'count')

,LGA_NAME,count
43,MELBOURNE,8430
13,CASEY,5348
26,GREATER GEELONG,4673
25,GREATER DANDENONG,4577
9,BRIMBANK,4257


### And the least:

In [100]:
accidents_by_lga.nsmallest(5,'count')

,LGA_NAME,count
60,QUEENSCLIFFE,24
78,YARRIAMBIACK,103
29,HINDMARSH,120
71,WEST WIMMERA,121
10,BULOKE,155


The finding that the least occured in Queenscliffe over this period should be fairly unsurprising, as at the 2016 Census, the Borough of Queenscliffe had only 2,853 residents!

### Number of accidents versus population by LGA

Obviously the above counts don't account for the size difference between places - many more people live in and drive through the City of Greater Geelong than do West Wimmera Shire.

Below, for each LGA, we calculate the proportion of crashes that happened per 100,000 residents as at the 2016 census. To make this number 'annual', we divide the result by 9.

This doesn't account for the boundary changes that may have happened to LGAs in that time.

In [102]:
df_lga_population = pd.read_csv(
    'abs_data/popn_by_lga_vic_census_2016.csv',
    names = ['LGA_NAME','popn'],
    skiprows=1 # first row is names, which we're setting manually
)

# make upper case to match vicroads data
df_lga_population['LGA_NAME'] = df_lga_population['LGA_NAME'].str.upper()

# merge into new data frame
accidents_by_lga_with_popn = pd.merge(accidents_by_lga, df_lga_population, on='LGA_NAME', how='left')

# rate per popn
accidents_by_lga_with_popn['crashes_per_100k_per_year'] = round(
    100000 * (
        accidents_by_lga_with_popn['count']/accidents_by_lga_with_popn['popn']/9
    ),0
)

In [105]:
seriesLGAPopnVsCrashesPer100K = go.Scatter(
    y = accidents_by_lga_with_popn['crashes_per_100k_per_year'],
    x = accidents_by_lga_with_popn['popn'],
    name = 'Crashes Per 100k Population vs. Population',
    mode = 'markers',
    text= accidents_by_lga_with_popn['LGA_NAME']
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents per 100,000 people vs. 2016 population',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        title = '2016 Population',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    yaxis = dict(
        title = 'Accidents per 100,000 people',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    legend=dict(
        orientation="h",
        x=.24,
        y=1.1
    ),
    annotations=[
        dict(
            x=135959,
            y=689,
            xref='x',
            yref='y',
            text='Melbourne LGA',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            ax=0,
            ay=-40
        ),
        dict(
            x=86675,
            y=397,
            xref='x',
            yref='y',
            text='Yarra LGA',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            ax=0,
            ay=-40
        )
    ]
)

data = [seriesLGAPopnVsCrashesPer100K]

figure8 = go.Figure(data=data, layout=layout)

### LGAs with less people have more accidents per person

* This is likely acting as a proxy for 'Country or not', and would align with the hypothesis that country roads are more dangerous than city roads
* What's going on with Melbourne and Yarra LGAs?
    * Is it perhaps the case minor accidents are more likely to be reported in the city?

In [106]:
py.iplot(figure8, filename='figure8')
#pyonline.image.ishow(figure8, width=1500, height=750)

### Are Melbourne and Yarra disproportionately likely to have minor accidents?


In [311]:
# group by both lga and severity
accidents_by_lga_by_severity = df_accident_node.groupby(
    ['LGA_NAME','SEVERITY']
).size()

# work out % of lga accidents
accidents_by_lga_by_severity_pc = accidents_by_lga_by_severity.groupby(level=0).apply(
    lambda x:round( 100 * x / float(x.sum()),2)
)

accidents_by_lga_by_severity_pc = accidents_by_lga_by_severity_pc.reset_index(name = 'pc')

# limit to just minor
accidents_by_lga_by_severity_minor = accidents_by_lga_by_severity_pc[accidents_by_lga_by_severity_pc['SEVERITY'] == 3]

# delete severity column
del accidents_by_lga_by_severity_minor['SEVERITY']

# sort by pc
accidents_by_lga_by_severity_minor = accidents_by_lga_by_severity_minor.sort_values(by='pc', ascending=True)

# make lower case
# run on the lga column of our df, add to new column
accidents_by_lga_by_severity_minor['LGA_NAME'] = accidents_by_lga_by_severity_minor['LGA_NAME'].apply(
    lambda x: x.title()
)

# rank by % minor
accidents_by_lga_by_severity_minor['rank'] = accidents_by_lga_by_severity_minor['pc'].rank(ascending=0)

### Not really, Melbourne and Yarra rank 10th and 11th for minor accidents:

In [294]:
accidents_by_lga_by_severity_minor[
    accidents_by_lga_by_severity_minor['rank'] <= 12
].sort_values(by='rank', ascending=True)

,LGA_NAME,pc,rank
149,Moonee Valley,71.10,1.0
8,Ballarat,70.83,2.0
95,Horsham,70.39,3.0
192,Stonnington,70.22,4.0
155,Moreland,69.65,5.0
11,Banyule,69.26,6.0
53,Darebin,68.49,7.0
225,Wodonga,68.23,8.0
107,Knox,68.13,9.0
131,Melbourne,67.58,10.0


### Is that higher accident rate in less populous LGAs a country thing?

In [203]:
# read from ABS Excel series
df_remoteness = pd.read_excel(
    'abs_data/1270055006_CG_LGA_2011_RA_2011.xls', # file name and path
    sheetname = 'Table 3',
    #header = 0,
    skiprows = 7,
    names = ['LGA_CODE_2011','LGA_NAME_2011','RA_CODE_2011','RA_NAME_2011','RATIO','PERCENTAGE']
)

# throw away values if not int (there is a copywrite logo in column A in the spreadsheet which breaks things)
df_remoteness = df_remoteness[df_remoteness['LGA_CODE_2011'].apply(lambda x: str(x).isdigit())]

# convert lga_code_2011 to int
df_remoteness['LGA_CODE_2011'] = df_remoteness['LGA_CODE_2011'].astype(int)

# remove < 20000 or >29999 (limit to vic, there is a kingston outside vic)
df_remoteness = df_remoteness[~(df_remoteness['LGA_CODE_2011'] < 20000)]
df_remoteness = df_remoteness[~(df_remoteness['LGA_CODE_2011'] >= 30000)]

## what's the value where percentage is at its maximum?
df_remoteness_main = df_remoteness.groupby(
    ['LGA_NAME_2011'],
    sort=False
)['PERCENTAGE'].max().reset_index(name='PERCENTAGE')

## merge to get a data frame of JUST maximum
df_remoteness_main = pd.merge(
    df_remoteness_main,
    df_remoteness,
    how='left',
    on=['LGA_NAME_2011','PERCENTAGE']
)

# drop redundant columns
del df_remoteness_main['RATIO']
del df_remoteness_main['PERCENTAGE']
del df_remoteness_main['RA_CODE_2011']

# lambda function = return all elemnts of string to the left of the first (, -1, then make it upper case
cleanup_lga = lambda x: x[:x.find('(')-1].upper()

# run on the lga column of our df, add to new column
df_remoteness_main['LGA_NAME'] = df_remoteness_main['LGA_NAME_2011'].apply(cleanup_lga)

print(df_remoteness_main.head(2))

  LGA_NAME_2011  LGA_CODE_2011              RA_NAME_2011 LGA_NAME
0    Alpine (S)          20110  Outer Regional Australia   ALPINE
1   Ararat (RC)          20260  Inner Regional Australia   ARARAT


In [204]:
accidents_by_lga_with_popn_with_remoteness = pd.merge(
    accidents_by_lga_with_popn,
    df_remoteness_main,
    how='left',
    on='LGA_NAME'
)

print(accidents_by_lga_with_popn_with_remoteness.head(2))

  LGA_NAME  count   popn  crashes_per_100k_per_year LGA_NAME_2011  \
0   ALPINE    478  12337                      431.0    Alpine (S)   
1   ARARAT    282  11600                      270.0   Ararat (RC)   

   LGA_CODE_2011              RA_NAME_2011  
0          20110  Outer Regional Australia  
1          20260  Inner Regional Australia  


In [300]:
series_cities = go.Scatter(
    y = accidents_by_lga_with_popn_with_remoteness['crashes_per_100k_per_year']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Major Cities of Australia'],
    x = accidents_by_lga_with_popn_with_remoteness['popn']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Major Cities of Australia'],
    name = 'City',
    mode = 'markers',
    text = accidents_by_lga_with_popn_with_remoteness['LGA_NAME']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Major Cities of Australia'],
    marker = dict (
        size = 10,
        opacity = 0.7
    )
)

series_ira = go.Scatter(
    y = accidents_by_lga_with_popn_with_remoteness['crashes_per_100k_per_year']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Inner Regional Australia'],
    x = accidents_by_lga_with_popn_with_remoteness['popn']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Inner Regional Australia'],
    name = 'Regional',
    mode = 'markers',
    text = accidents_by_lga_with_popn_with_remoteness['LGA_NAME']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Inner Regional Australia'],
    marker = dict (
        size = 10,
        opacity = 0.7
    )
)

series_ora = go.Scatter(
    y = accidents_by_lga_with_popn_with_remoteness['crashes_per_100k_per_year']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Outer Regional Australia'],
    x = accidents_by_lga_with_popn_with_remoteness['popn']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Outer Regional Australia'],
    name = 'Rural',
    mode = 'markers',
    text = accidents_by_lga_with_popn_with_remoteness['LGA_NAME']
        [accidents_by_lga_with_popn_with_remoteness['RA_NAME_2011'] == 'Outer Regional Australia'],
    marker = dict (
        size = 10,
        opacity = 0.7
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents per 100,000 people vs. 2016 population',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        title = '2016 Population',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    yaxis = dict(
        title = 'Accidents per 100,000 people',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    legend=dict(
        orientation="h",
        x=.35,
        y=1.1
    ),
    annotations=[
        dict(
            x=101686,
            y=265,
            xref='x',
            yref='y',
            text='Ballarat',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            ax=0,
            ay=-40
        ),
        dict(
            x=110477,
            y=260,
            xref='x',
            yref='y',
            text='Bendigo',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            ax=0,
            ay=40
        )
    ]
)

data = [series_ora, series_ira, series_cities]

figure9 = go.Figure(data=data, layout=layout)

### Accidents by LGA by ABS Remoteness Area

* The chart below colours LGAs by the ABS Remoteness Area they fall most within
    * Regional is the label used for Inner Regional Australia
    * Rural is used for Outer Regional Australia

### This is a nice visual way to confirm what the earlier chart told us - per capita, more accidents happen in the country

* Interestingly, the two 'regional' councils with big populations, those based around Bendigo and Ballarat, fall in line with those of Melbourne's suburbs, we are seeing the effect of large parts of these LGAs being cities

In [301]:
py.iplot(figure9, filename='figure9')
#pyonline.image.ishow(figure9, width=1500, height=750)

### Does the pattern with the country repeat for deaths?


In [323]:
# make df of just lethal accidents
accidents_by_lga_by_severity_mod = accidents_by_lga_by_severity.reset_index(name='count')
accidents_by_lga_lethal = accidents_by_lga_by_severity_mod[accidents_by_lga_by_severity_mod['SEVERITY'] == 1]
del accidents_by_lga_lethal['SEVERITY']

# merge in population
accidents_by_lga_lethal_with_popn = pd.merge(accidents_by_lga_lethal, df_lga_population, on='LGA_NAME', how='left')

# calculate figures per 100k
accidents_by_lga_lethal_with_popn['crashes_per_100k_per_year'] = round(
    100000 * (
        accidents_by_lga_lethal_with_popn['count']/accidents_by_lga_with_popn['popn']/9
    ),1
)

# merge in remoteness
accidents_by_lga_lethal_with_popn_with_remoteness = pd.merge(
    accidents_by_lga_lethal_with_popn,
    df_remoteness_main,
    how='left',
    on='LGA_NAME'
)

In [341]:
series_cities = go.Scatter(
    y = accidents_by_lga_lethal_with_popn_with_remoteness['crashes_per_100k_per_year']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Major Cities of Australia'],
    x = accidents_by_lga_lethal_with_popn_with_remoteness['popn']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Major Cities of Australia'],
    name = 'City',
    mode = 'markers',
    text = accidents_by_lga_lethal_with_popn_with_remoteness['LGA_NAME']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Major Cities of Australia'],
    marker = dict (
        size = 10,
        opacity = 0.7
    )
)

series_ira = go.Scatter(
    y = accidents_by_lga_lethal_with_popn_with_remoteness['crashes_per_100k_per_year']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Inner Regional Australia'],
    x = accidents_by_lga_lethal_with_popn_with_remoteness['popn']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Inner Regional Australia'],
    name = 'Regional',
    mode = 'markers',
    text = accidents_by_lga_lethal_with_popn_with_remoteness['LGA_NAME']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Inner Regional Australia'],
    marker = dict (
        size = 10,
        opacity = 0.7
    )
)

series_ora = go.Scatter(
    y = accidents_by_lga_lethal_with_popn_with_remoteness['crashes_per_100k_per_year']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Outer Regional Australia'],
    x = accidents_by_lga_lethal_with_popn_with_remoteness['popn']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Outer Regional Australia'],
    name = 'Rural',
    mode = 'markers',
    text = accidents_by_lga_lethal_with_popn_with_remoteness['LGA_NAME']
        [accidents_by_lga_lethal_with_popn_with_remoteness['RA_NAME_2011'] == 'Outer Regional Australia'],
    marker = dict (
        size = 10,
        opacity = 0.7
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Lethal accidents per 100,000 people vs. 2016 population',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        title = '2016 Population',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    yaxis = dict(
        title = 'Lethal accidents per 100,000 people',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    ),
    legend=dict(
        orientation="h",
        x=.35,
        y=1.1
    ),
    annotations=[
        dict(
            x=10274,
            y=34.6,
            xref='x',
            yref='y',
            text='Strathbogie',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            ax=75,
            ay=0
        ),
        dict(
            x=299301,
            y=3.2,
            xref='x',
            yref='y',
            text='Casey',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            ax=0,
            ay=-40
        )
    ]
)

data = [series_ora, series_ira, series_cities]

figure10 = go.Figure(data=data, layout=layout)

### Lethal accidents per 100k persons, versus total persons

* This relationship is even stronger than that for accidents in general
* Per capita, many more people die on country roads in Victoria than those in the cities
* Per capita, 10 times as many lethal accidents occur on the roads of the Strathbogie LGA as do on those of the Casey LGA

In [340]:
py.iplot(figure10, filename='figure10')
#pyonline.image.ishow(figure10, width=1500, height=750)

# Vehicle statistics

In [354]:
df_vehicle = pd.read_csv(
    'vicroads_data/ACCIDENT/VEHICLE.csv', # file name and path
    sep=',', # separated by commas
    low_memory = False
)

In [351]:
colours = df_vehicle.groupby('VEHICLE_COLOUR_1').size().reset_index(name='count')

df_colour_labels = pd.read_csv(
    'colours.csv',
    sep=','
)

colours = pd.merge(colours, df_colour_labels, on='VEHICLE_COLOUR_1', how='left')

del colours['VEHICLE_COLOUR_1']

### What colour cars are most frequently involved in crashes?

In [353]:
colours.sort_values(by='count',ascending=False).head(5)

,count,colour
15,61342,White
14,41002,Silver
1,35507,Blue
0,28473,Black
13,27880,Red


# Person statistics

In [396]:
df_person = pd.read_csv(
    'vicroads_data/ACCIDENT/PERSON.csv', # file name and path
    sep=',', # separated by commas
    low_memory = False
)

drivers = ['Drivers','Motorcyclists']

df_person_drivers = df_person[df_person['Road User Type Desc'].isin(drivers)]
print(df_person_drivers)

df_person_notdrivers = df_person[ ~(df_person['Road User Type Desc'].isin(drivers) )]

         ACCIDENT_NO PERSON_ID VEHICLE_ID SEX   AGE Age Group  INJ_LEVEL  \
3       T20060000010        A           A   M  72.0       70+          4   
4       T20060000010        B           B   F  62.0     60-64          3   
5       T20060000010        C           C   M  39.0     30-39          4   
8       T20060000018        A           A   M  30.0     30-39          4   
9       T20060000018        B           B   F  20.0     17-21          3   
11      T20060000022        A           A   M  45.0     40-49          4   
12      T20060000023        A           A   M  43.0     40-49          2   
13      T20060000023        B           B   M  42.0     40-49          4   
15      T20060000026        A           A   M  84.0       70+          3   
16      T20060000026        B           B   M  51.0     50-59          3   
17      T20060000028        A           A   M  85.0       70+          4   
18      T20060000028        B           B   M  57.0     50-59          2   
19      T200

### Ages

In [395]:
age_all_people = df_person.groupby('Age Group').size().reset_index(name='all')
age_drivers = df_person_drivers.groupby('Age Group').size().reset_index(name='drivers')

age = pd.merge(age_all_people, age_drivers, how='left', on='Age Group').sort_values(by='Age Group')
print(age)

   Age Group    all  drivers
0        0-4   8040        4
1      13-15   6779      257
2      16-17   8118     1423
3      17-21  43341    29018
4      22-25  37522    26652
5      26-29  31465    23059
6      30-39  63851    49117
7      40-49  55493    44033
8       5-12  13528       47
9      50-59  41947    32679
10     60-64  15012    11284
11     64-69  10714     7643
12       70+  21851    14398
13   unknown  16850     7934


In [397]:
series_all_people = go.Bar(
    y = age['all'],
    x = age['Age Group'],
    name = 'All People',
    marker = dict(
        color = '#FF6900'
    )
)

series_drivers = go.Bar(
    y = age['drivers'],
    x = age['Age Group'],
    name = 'Drivers',
    marker = dict(
        color = '#0077C8'
    )
)

layout = go.Layout(
    barmode = 'group',
    title = 'VicRoads Accident - Accidents by Hour',
    titlefont=dict(
            family='Open Sans',
            size=22
    ),
    xaxis=dict(
        #tickangle = -45, # angle lables at 45 deg
        dtick = 1,
        title = 'Age Ranges',
            titlefont=dict(
                family='Open Sans',
                size=16
            )
    ),
    yaxis = dict(
        title = 'Number of People',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
    )
)

data = [series_all_people, series_drivers]

figure11 = go.Figure(data=data, layout=layout)

### Older people are more likely to be drivers, thoroughly unsurprisingly

In [398]:
py.iplot(figure11, filename='figure10')
#pyonline.image.ishow(figure10, width=1500, height=750)